# Introduction

This script trains a model which has a ConvLSTM2D layer to encode temporal and spatial information of a video for classification of mood of person depending on gait of the person. 

Input to model: Sequence of images of person extracted from video (Video strictly contains a person walking towards the camera in frontal view showing his/her full body).

Output from model: Probability of a person being in class 1: happy.

## Before Running Program

Move the videos used for training and testing and store in their respective folder named: "train" and "test" created in base directory
**IMPORTANT** video files are named in the format: "VID_RGB_xxx_y.mp4" where "xxx" is a unique index of the video file and "y" is the label of the file where 0: depressed, 1: healthy

## Set variables for training of model

In [11]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

SEQ_SIZE = 30
MOBILE = False #set to true if input videos are recorded from mobile phone
IMG_WIDTH, IMG_HEIGHT = 100, 700 #size of images to be taken in by the ConvLSTM2D layer.
EPOCHS = 17
FILTER_SIZE = 20 #filter size of the ConvLSTM2D layer
LEARNING_RATE = 1e-5
KERNEL_SIZE=3 #kernel size of the ConvLSTM2D layer
BATCH_SIZE=3
LOGS = "logs" #set the directory to store logs to be viewed with tensorboard if needed
WEIGHTS_DIR = "weights" #set the directory where checkpoints of model can be saved to if needed
EARLY_STOP = False
SAVE_MODEL = True #set to true if want to save model
DEBUG = True #set to true when debugging to print out progress during training 

### 1.  Perform DeepSORT on input videos.

Set up default folder path. This is fixed, do not change the file structure.

In [2]:
import os, sys
from os.path import exists, join, basename
from pathlib import Path
from fyp_train_gen_img_model import train_model
BASE_DIR = os.getcwd()
called_dir = BASE_DIR
while os.path.basename(BASE_DIR) != "fyp_team4c":
    path = Path(BASE_DIR)
    BASE_DIR = str(path.parent)
    if BASE_DIR == '/':
        print("Please call this script in the fyp_team4c directory")
        break
sys.path.append(BASE_DIR)
from utils import *

TRAINING_DIR = os.path.join(BASE_DIR, 'training')
VIDEO_DIRNAMES = ["train", "test"]



Clearing session


Run DeepSort on videos in "train" and "test"

In [4]:
output_dir = os.path.join(TRAINING_DIR, 'output')
mobile = True
for vid_dirname in VIDEO_DIRNAMES:
    videos_dir = os.path.join(TRAINING_DIR, vid_dirname)
    vid_l = [f for f in sorted(os.listdir(videos_dir)) if f.endswith(".mp4")]
    for fn in vid_l:
        input_vid_p = os.path.join(videos_dir, fn)
        run_deepsort(output_dir, input_vid_p, mobile, called_dir, BASE_DIR)


/home/student/jiawen/project/fyp_team4c/deep_sort_pytorch
/home/student/jiawen/project/fyp_team4c/training/output/final_vid
/home/student/jiawen/project/fyp_team4c/training/output/bbox_output
/home/student/jiawen/project/fyp_team4c/training/train
Checking if FINAL_VID_DIR exist /home/student/jiawen/project/fyp_team4c/training/output/final_vid
Checking if BBOX_DIR exist /home/student/jiawen/project/fyp_team4c/training/output/bbox_output
/home/student/jiawen/project/fyp_team4c/deep_sort_pytorch
/home/student/jiawen/project/fyp_team4c/training/output/final_vid
/home/student/jiawen/project/fyp_team4c/training/output/bbox_output
/home/student/jiawen/project/fyp_team4c/training/train
/home/student/jiawen/project/fyp_team4c/deep_sort_pytorch
/home/student/jiawen/project/fyp_team4c/training/output/final_vid
/home/student/jiawen/project/fyp_team4c/training/output/bbox_output
/home/student/jiawen/project/fyp_team4c/training/train
/home/student/jiawen/project/fyp_team4c/deep_sort_pytorch
/home/st

/home/student/jiawen/project/fyp_team4c/deep_sort_pytorch
/home/student/jiawen/project/fyp_team4c/training/output/final_vid
/home/student/jiawen/project/fyp_team4c/training/output/bbox_output
/home/student/jiawen/project/fyp_team4c/training/test


### 2. Extract frames of videos in "train" and "test" folder and save it to the "frames" folder

In [6]:
import os
#create the frames folder
frames_folder = os.path.join(output_dir, 'frames')

        
for vid_dirname in VIDEO_DIRNAMES:
    vid_dir = os.path.join(TRAINING_DIR, vid_dirname)
    vid_frame_dir = os.path.join(frames_folder, vid_dirname)
    if not os.path.exists(vid_frame_dir):
        os.makedirs(vid_frame_dir)
        print("Creating directory: ", vid_frame_dir)
        
    for fn in sorted(os.listdir(vid_dir)):
        if fn[-3:] == "mp4":
            vid_fp = os.path.join(vid_dirname, fn)
            print(f'Extracting to {vid_dirname}: ',fn)
            extract_frames(vid_fp, fn, vid_frame_dir)

Creating directory:  /home/student/jiawen/project/fyp_team4c/training/output/frames/train
Extracting to train:  VID_RGB_0010_1.mp4
Extracting  train/VID_RGB_0010_1.mp4 to  /home/student/jiawen/project/fyp_team4c/training/output/frames/train
Extracting to train:  VID_RGB_0011_1.mp4
Extracting  train/VID_RGB_0011_1.mp4 to  /home/student/jiawen/project/fyp_team4c/training/output/frames/train
Extracting to train:  VID_RGB_0012_1.mp4
Extracting  train/VID_RGB_0012_1.mp4 to  /home/student/jiawen/project/fyp_team4c/training/output/frames/train
Extracting to train:  VID_RGB_0013_1.mp4
Extracting  train/VID_RGB_0013_1.mp4 to  /home/student/jiawen/project/fyp_team4c/training/output/frames/train
Extracting to train:  VID_RGB_0014_1.mp4
Extracting  train/VID_RGB_0014_1.mp4 to  /home/student/jiawen/project/fyp_team4c/training/output/frames/train
Extracting to train:  VID_RGB_0015_1.mp4
Extracting  train/VID_RGB_0015_1.mp4 to  /home/student/jiawen/project/fyp_team4c/training/output/frames/train
Extr

### 3. Extract bounding box of each detected frame located in /frames/train and /frames/test/ and save each extracted box to a folder named with their respective detected ID.

In [7]:
bbox_dir = os.path.join(output_dir, 'bbox_output')

for fn in sorted(os.listdir(bbox_dir)):
    #fn is in format {filename}_bbox.pkl
    vid_fn = f'{fn[:-9]}.mp4'

    if fn[-4:] == ".pkl":
        input_frames_folder = ""
        vid_exists = False
    for vid_dirname in VIDEO_DIRNAMES:
        vid_dir = os.path.join(TRAINING_DIR, vid_dirname)
        vid_fp = os.path.join(vid_dir, vid_fn)
        if os.path.exists(vid_fp):
#             input_frames_folder = f'{FRAMES_FOLDER}/{vid_dirname}'
            input_frames_folder = os.path.join(frames_folder, vid_dirname)
            vid_exists=True
            break
    if not vid_exists:
        continue
    print(f"Extracting {fn} from {input_frames_folder}")
    bbox_p = os.path.join(bbox_dir, fn)
    extract_bbox(vid_fn[:-4], bbox_p, input_frames_folder)

Extracting VID_RGB_0010_1_bbox.pkl from /home/student/jiawen/project/fyp_team4c/training/output/frames/train
Extracting VID_RGB_0011_1_bbox.pkl from /home/student/jiawen/project/fyp_team4c/training/output/frames/train
Extracting VID_RGB_0012_1_bbox.pkl from /home/student/jiawen/project/fyp_team4c/training/output/frames/train
Extracting VID_RGB_0013_1_bbox.pkl from /home/student/jiawen/project/fyp_team4c/training/output/frames/train
Extracting VID_RGB_0014_1_bbox.pkl from /home/student/jiawen/project/fyp_team4c/training/output/frames/train
Extracting VID_RGB_0015_1_bbox.pkl from /home/student/jiawen/project/fyp_team4c/training/output/frames/train
Extracting VID_RGB_0016_1_bbox.pkl from /home/student/jiawen/project/fyp_team4c/training/output/frames/train
Extracting VID_RGB_0017_1_bbox.pkl from /home/student/jiawen/project/fyp_team4c/training/output/frames/train
Extracting VID_RGB_001_1_bbox.pkl from /home/student/jiawen/project/fyp_team4c/training/output/frames/test
Extracting VID_RGB_00

### 4. Train the model with the parameters set previously.

In [12]:
train_model(IMG_WIDTH, IMG_HEIGHT, 'pickled_seq_images', lr=LEARNING_RATE,batch_size=BATCH_SIZE, epochs=EPOCHS, seq_size=SEQ_SIZE, filter_size=FILTER_SIZE, debug=DEBUG, logs=LOGS, weights_dir=WEIGHTS_DIR,save_model=SAVE_MODEL,early_stop=EARLY_STOP)

72 23 HALOOOOOOOOOOOOO
Creating model2
Compiling model
Fitting model with batch size:  3
Epoch 1/17
Tensor("Cast_2:0", shape=(None, 1), dtype=float32) Tensor("sequential_5/dense_17/Sigmoid:0", shape=(None, 1), dtype=float32)
Tensor("Cast_2:0", shape=(None, 1), dtype=float32) Tensor("sequential_5/dense_17/Sigmoid:0", shape=(None, 1), dtype=float32)
24/24 [==============================] - 30s 1s/step - loss: 0.3146 - get_f1: 0.5847 - val_loss: 0.7003 - val_get_f1: 0.0000e+00
Epoch 2/17
24/24 [==============================] - 29s 1s/step - loss: 0.2670 - get_f1: 0.7097 - val_loss: 0.7011 - val_get_f1: 0.0000e+00
Epoch 3/17
24/24 [==============================] - 29s 1s/step - loss: 0.2045 - get_f1: 0.7458 - val_loss: 0.6995 - val_get_f1: 0.0000e+00
Epoch 4/17
24/24 [==============================] - 29s 1s/step - loss: 0.2020 - get_f1: 0.7903 - val_loss: 0.6963 - val_get_f1: 0.0000e+00
Epoch 5/17
24/24 [==============================] - 29s 1s/step - loss: 0.1768 - get_f1: 0.8417 - val